'''
Example script to automatically write a story
Demonstrates:
- multiple API endpoints (offical VERTEX AI)
- running single tasks: spam detection and scoring
- running a crew to create a screenplay from a newsgroup post by first analyzing the text, creating a dialogue and ultimately formatting it
Additional endpoints requirements:
  pip install langchain_VERTEXAI
Author: SAURABH MANGAL
License: MIT
'''

In [1]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun


In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part



def generate_pro(input_prompt):
    model = GenerativeModel("gemini-1.0-pro-002") #gemini-1.5-pro-preview-0409
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    # print (responses.text)
    
    return(responses.text)


# input_prompt = '''
# Action 'generateimage(chapter_content_and_character_details={'chapter_content': 'Panic ensued in the animal kingdom as Professor Hoot, the wise old owl, was called upon to investigate. The mystery deepened as they searched for clues to unravel the enigma of Zeke's missing stripes.', 'character_details': 'Professor Hoot is a wise old owl'})' don't exist, these are the only available Actions: generateimage: generateimage(chapter_content_and_character_details: 'string') -      Generates an image using Imagen for a given chapter content and character details.     

# '''

# generate_pro(input_prompt)

In [4]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name="gemini-1.0-pro-002", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)

# Initialize Imagen model

@tool
def generateimage(chapter_content_and_character_details: str) -> str:
    """
    Generates an image using Imagen for a given chapter content and character details.
    """
    image_generation_model = ImageGenerationModel.from_pretrained("imagegeneration@006")

    chapter_content_and_character_details_refined = generate_pro(chapter_content_and_character_details)
    
    prompt = f"Image is about: {chapter_content_and_character_details_refined}. Style: Illustration. Create an illustration incorporating a vivid palette with an emphasis on shades of azure and emerald, augmented by splashes of gold for contrast and visual interest. The style should evoke the intricate detail and whimsy of early 20th-century storybook illustrations, blending realism with fantastical elements to create a sense of wonder and enchantment. The composition should be rich in texture, with a soft, luminous lighting that enhances the magical atmosphere. Attention to the interplay of light and shadow will add depth and dimensionality, inviting the viewer to delve into the scene. DON'T include ANY text in this image. DON'T include colour palettes in this image."

    response = image_generation_model.generate_images(
        prompt=prompt,
        aspect_ratio="1:1",
        number_of_images=1,
        safety_filter_level="block_few",
        person_generation="allow_adult",
    )
    
    image_url_name = str(uuid.uuid4())
    
    filepath = os.path.join(os.getcwd(), "images2/")

    image_url = filepath+image_url_name+".jpg"
    response.images[0].save(location=image_url)
    print(image_url)

    return(image_url)

In [12]:
@tool("Search the internet")
def latest_trends(query):
    """Searches the internet for a given topic and returns relevant results."""
    url = f"https://www.google.com/search?q={query}"
    #response = requests.get(url)
    # Create a list to store the results
    results = []
    for j in search(query, tld="com", num=10, stop=10, pause=2):
        results.append(j)
    return results

search_tool = DuckDuckGoSearchRun()



In [13]:
# ... early imports, tools, setting OpenAI API key, etc
from crewai import Agent, Task, Crew
from crewai_tools.tools import FileReadTool

file_read_tool = FileReadTool(
    file_path='template.md',
    description='A tool to read the Story Template file and understand the expected output format.'
)

# _________________ Agent DEFINATION __________________

idea_analyst = Agent(
    role='Story Outliner',
    goal='Develop an outline for a children\'s storybook about Animals, including chapter titles and characters for 5 chapters.',
    backstory="An imaginative creator who lays the foundation of captivating stories for children.",
    verbose=True,
    llm=llm,
    allow_delegation=False,
    tools=[search_tool]
)

content_formatter = Agent(
    role='Content Formatter',
    goal='Format the written story content in markdown, including images at the beginning of each chapter and all full story line based on the content.',
    backstory='A meticulous formatter who enhances the readability and presentation of the storybook.',
    verbose=True,
    llm=llm,
    tools=[file_read_tool, generateimage],
    allow_delegation=False
)


image_generator = Agent(
  role='Image Generator',
  goal='Generate one image per chapter content provided by the story outliner. Start with Chapter number, chapter content, character details, detailed location information and detailed items in the location where the activity happens. Generate totally 5 images one by one. Final output should contain all the 5 images in json format.',
  backstory="A creative AI specialized in visual storytelling, bringing each chapter to life through imaginative imagery.",
  verbose=True,
  llm=llm,
  tools=[generateimage],
  allow_delegation=False
)

# _________________ TASK DEFINATION __________________

expand_idea_task = Task(
    description='Convert a Markdown file to a PDF document, ensuring the preservation of formatting, structure, and embedded images using the mdpdf library.',
    agent=idea_analyst,
    expected_output='A PDF file generated from the Markdown input, accurately reflecting the content with proper formatting. The PDF should be ready for sharing or printing.'
)

task_image_generate = Task(
    description='Generate 5 images that captures the essence of the children\'s storybook about Animals, aligning with the themes, characters, and narrative outlined for the chapters. Do it one by one.',
    agent=image_generator,
    expected_output='A digital image file that visually represents the overarching theme of the children\'s storybook, incorporating elements from the characters and plot as described in the outline. The image should be suitable for inclusion in the storybook as an illustration.',
)

# refine_idea_task = Task(...) another task

## Create the crew that will produce a fully formed idea proposal
crew = Crew(
 agents=[idea_analyst,image_generator, content_formatter],
 tasks=[expand_idea_task, task_image_generate],
 verbose=True
)

final_improved_idea = crew.kickoff() # returns the final idea proposal

2024-05-03 00:23:11,175 - 140149537371968 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Story Outliner
 [INFO]: == Starting Task: Convert a Markdown file to a PDF document, ensuring the preservation of formatting, structure, and embedded images using the mdpdf library.


> Entering new CrewAgentExecutor chain...
## Children's Storybook Outline: Animal Adventures

**Theme:** Friendship, teamwork, and appreciating individual strengths.

**Characters:**

* **Max the Monkey:** Curious, playful, and quick-witted.
* **Pip the Panda:** Gentle, kind, and strong.
* **Ollie the Owl:** Wise, observant, and knowledgeable.
* **Zippy the Zebra:** Energetic, optimistic, and fast.
* **Echo the Elephant:** Calm, helpful, and reliable.

**Chapter Titles:**

**Chapter 1: The Mysterious Message**

- Max finds a strange object in the jungle – a bottle with a rolled-up message inside.
- Pip, Ollie, Zippy, and Echo join Max to decipher the message, which leads them on a treasure hunt.

**Chapter 2: Through the Whispering Woods**

- The animal friends follow clues thr